In [1]:
import glob
import sys
import numpy as np

from enterprise.pulsar import Pulsar
from enterprise.signals import signal_base
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals

import dpdm_delay

# import customized files to introduce 2T modes.
import gp_bases_new as gp_bases
import pl_prior as gp_priors

from enterprise.signals import selections

import multiprocess
from PTMCMCSampler.PTMCMCSampler import PTSampler


You'll need sksparse for get_coefficients() with common signals!
Cannot import PINT? Meh...
Do not have mpi4py package.


In [2]:
datadir = 'DR2'

parfiles = sorted(glob.glob(datadir + '/newpars/*.par'))
timfiles = sorted(glob.glob(datadir + '/toas/*.tim'))

In [3]:
psrs=[]
for ipsr in range(len(parfiles)):
    psr = Pulsar(parfiles[ipsr], timfiles[ipsr])
    psrs.append(psr)
    print(psr.name)

J0437-4715
J0613-0200
J0711-6830


J1017-7156
J1022+1001
J1024-0719
J1045-4509


J1125-6014


J1446-4701


J1545-4550
J1600-3053
J1603-7202
J1643-1224
J1713+0747
J1730-2304
J1732-5049
J1744-1134


J1824-2452A
J1832-0836
J1857+0943
J1909-3744
J1939+2134
J2124-3358
J2129-5721
J2145-0750


J2241-5236


In [4]:
# red noise
nmodes = 30


log10_A = parameter.Constant()
gamma = parameter.Constant()
pl = gp_priors.powerlaw(log10_A=log10_A, gamma=gamma)
dm_basis = gp_bases.createfourierdesignmatrix_dm(nmodes=nmodes)
red_basis = gp_bases.createfourierdesignmatrix_red(nmodes=nmodes)
selection = selections.Selection(selections.by_band)

dmn = gp_signals.BasisGP(pl, dm_basis, name='dm', coefficients=False)
spn = gp_signals.BasisGP(pl, red_basis, name='sp',coefficients=False)
bdn = gp_signals.BasisGP(pl, red_basis, name='bd',coefficients=False,selection=selection)

# white noise
backend = selections.Selection(selections.by_backend)
efac = parameter.Constant()
equad = parameter.Constant()

ef = white_signals.MeasurementNoise(efac=efac,selection=backend)
eq = white_signals.EquadNoise(log10_equad=equad,selection=backend)
wnb = ef + eq

# timing model
tm = gp_signals.TimingModel()

# Bayes ephmeris
eph = deterministic_signals.PhysicalEphemerisSignal(use_epoch_toas=True)


# include best fit parameters
Dict = np.load('noise_params/noisepars_m'+str(nmodes)+'.npy',allow_pickle=True,encoding='bytes').item()
len(Dict)

638

## Prepare for Bayes Limits

In [5]:
def single_task(log10_ma, name, ret, model, test=False):
    
        np.random.seed()
    
        log10_eps = parameter.Uniform(-28.0, -16.0)('x_dp_log10_eps')
        dp = dpdm_delay.dpdm_block(model=model,log10_ma=log10_ma,log10_eps=log10_eps)

        model0  = tm + wnb + dmn + spn + eph + dp
        model1  = tm + wnb + dmn + spn + eph + dp + bdn

        signal=[]
        for psr in psrs:
            if psr.name in ['J0437-4715','J1939+2134']:
                signal.append( model1( psr ) )
            else:
                signal.append( model0( psr ) )

        pta = signal_base.PTA(signal)
        pta.set_default_params(Dict)
        if test == True:
            return pta

        #outDir="bayes/"+model+"_m"+str(nmodes)+"/n"+str(i).zfill(3)+'/'
        outDir="bayes/"+model+"_m"+str(nmodes)+'/'+name+'/'
#============================
        # to make the initial values
        xs = {par.name:par.sample() for par in pta.params}
        for parname in Dict.keys():
            if parname in xs.keys():
                xs[parname] = Dict[parname]
        
        x0 = np.hstack([xs[key] for key in sorted(xs.keys())])    
        ndim = len(x0)
#============================

        x0[-19:-8] = 0.
        x0[-8] = -26.
        
        groups=[range(ndim),range(ndim-8,ndim)]
        
        print(pta.get_lnlikelihood(x0))
        
        
        covdiag = np.ones(ndim)*0.01

        sampler = PTSampler(ndim, pta.get_lnlikelihood, pta.get_lnprior,
                        cov=np.diag(covdiag),outDir=outDir,groups=groups,resume=False)

        sampler.sample(x0, 40000, isave=1000)
        
        #chain=np.loadtxt(outDir+'/chain_1.txt')

        #logep=chain[int(0.8*len(chain)):,-12]

        #ret.value = np.percentile(logep,95)


## Run the code

In [11]:
for names in ["C","D","E"]:
    for j in range(7):
        a = 0 + 12*j
        b = 12 + 12*j

        print(a,b,nmodes)

        jobs = []
        RETs={}

        for i in range(a,b): 

            RETs[i] = multiprocess.Manager().Value('i',0)

            logmamin = -23.5 + 2.5 * float(i) / float(125)
            logmamax = -23.5 + 2.5 * float(i+1) / float(125)

            log10_ma = parameter.Uniform(logmamin,logmamax)('x_dp_log10_ma')#n=0~99

            p = multiprocess.Process(target=single_task, args=(log10_ma, names+str(i).zfill(3) , RETs[i] , "U1BL" ))

            jobs.append(p)

            p.start()


        for p in jobs:

            p.join()
    


0 12 30
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L
model:U1B-L


INFO: enterprise.signals.signal_base: Setting J0437-4715_CASPSR_40CM_efac to 1.1821247662122054
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_10CM_efac to 1.0975825129925123
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_20CM_efac to 1.0248557925516382
INFO: enterprise.signals.signal_base: Setting J0437-4715_CPSR2_50CM_efac to 1.0204302047551015
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_10CM_efac to 1.010201435266573
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_1433_efac to 0.22891987956193693
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_20CM_efac to 0.9537635425047156
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_early_10CM_efac to 0.6822931094912945
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB1_early_20CM_efac to 0.4683737710247763
INFO: enterprise.signals.signal_base: Setting J0437-4715_PDFB_10CM_efac to 0.11937728681829984
INFO: enterprise.signals.signal_base:

INFO: enterprise.signals.signal_base: Setting J0613-0200_WBCORR_10CM_efac to 0.9826384706095648
INFO: enterprise.signals.signal_base: Setting J0437-4715_bd_50CM_log10_A to -12.464417096558837
INFO: enterprise.signals.signal_base: Setting J0613-0200_CASPSR_40CM_log10_equad to -6.56313547089816
INFO: enterprise.signals.signal_base: Setting J0437-4715_bd_50CM_gamma to 4.428293763992251
INFO: enterprise.signals.signal_base: Setting J0613-0200_CPSR2_20CM_log10_equad to -8.287919320282041
INFO: enterprise.signals.signal_base: Setting J0613-0200_CPSR2_50CM_log10_equad to -6.397413428213118
INFO: enterprise.signals.signal_base: Setting J0613-0200_CASPSR_40CM_efac to 1.5121762821219427
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_10CM_log10_equad to -5.767366597456407
INFO: enterprise.signals.signal_base: Setting J0613-0200_CPSR2_20CM_efac to 1.0171079701752663
INFO: enterprise.signals.signal_base: Setting J0613-0200_PDFB1_1433_log10_equad to -6.31395559493206
INFO: enterprise

INFO: enterprise.signals.signal_base: Setting J0711-6830_sp_gamma to 0.6265684246369604
INFO: enterprise.signals.signal_base: Setting J0711-6830_PDFB_40CM_efac to 0.9793178133299563
INFO: enterprise.signals.signal_base: Setting J1017-7156_CASPSR_40CM_efac to 0.8588849502216462
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_10CM_efac to 0.9796248949731852
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_20CM_efac to 0.979985298841725
INFO: enterprise.signals.signal_base: Setting J0711-6830_WBCORR_10CM_efac to 0.9739499561717543
INFO: enterprise.signals.signal_base: Setting J1017-7156_PDFB_40CM_efac to 1.0209923398923604
INFO: enterprise.signals.signal_base: Setting J0711-6830_CASPSR_40CM_log10_equad to -7.663005915869408
INFO: enterprise.signals.signal_base: Setting J1017-7156_CASPSR_40CM_log10_equad to -5.953116628703558
INFO: enterprise.signals.signal_base: Setting J0711-6830_CPSR2_20CM_log10_equad to -5.940468437823865
INFO: enterprise.signals.signal_base:

INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB1_early_20CM_efac to 0.9436496680969977
INFO: enterprise.signals.signal_base: Setting J1024-0719_PDFB1_early_20CM_efac to 0.943120007221042
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB_10CM_efac to 0.9159529702871555
INFO: enterprise.signals.signal_base: Setting J1024-0719_PDFB_10CM_efac to 0.9893620462086686
INFO: enterprise.signals.signal_base: Setting J1024-0719_PDFB_20CM_efac to 1.0018523096695922
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB_20CM_efac to 0.9842836863868607
INFO: enterprise.signals.signal_base: Setting J1022+1001_PDFB_40CM_efac to 1.0425094562483648
INFO: enterprise.signals.signal_base: Setting J1024-0719_PDFB_40CM_efac to 1.0001296198846015
INFO: enterprise.signals.signal_base: Setting J1022+1001_WBCORR_10CM_efac to 1.0080350482402987
INFO: enterprise.signals.signal_base: Setting J1024-0719_WBCORR_10CM_efac to 0.964653292980362
INFO: enterprise.signals.signal_base: Settin

...

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-7563ac3ff8be>", line 49, in single_task
    sampler.sample(x0, 40000, isave=1000)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/PTMCMCSampler-2015.2-py3.7.egg/PTMCMCSampler/PTMCMCSampler.py", line 362, in sample
    p0, lnlike0, lnprob0, iter)
  File "/home/xuex/anaconda3/

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 262, in <listcomp>
    return [signalcollection.get_TNr(params) for signalcollection in self._signalcollections]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
    cache[key] = func(self, params)
KeyboardInterrupt
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 833, in get_TNr
    res = self.get_detres(params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/PTMCMCSampler-2015.2-py3.7.egg/PTMCMCSampler/PTMCMCSampler.py", line 490, in PTMCMCOneStep
    newlnlike = self.logl(y)
  File "<__array_function__ internals>", line 6, in sum
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py", line 2175, in sum
    res = _sum_(a)
Process Process-88:
Process Process-9

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 169, in __call__
    TNrs = self.pta.get_TNr(params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 262, in get_TNr
    return [signalcollection.get_TNr(params) for signalcollection in self._signalcollections]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
    cache[key] = func(self, params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 

  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 802, in get_detres
    return self._residuals - self.get_delay(params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 900, in wrapper
    cache[key] = func(self, params)
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 262, in get_TNr
    return [signalcollection.get_TNr(params) for signalcollection in self._signalcollections]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 797, in get_delay
    delays = [signal.get_delay(params) for signal in self._signals]
  File "/home/xuex/anaconda3/lib/python3.7/site-packages/enterprise-1.2.1-py3.7.egg/enterprise/signals/signal_base.py", line 797, in <listcomp>
    delays = [signal.get_delay(params

Finished 62.50 percent in 2407.713552 s Acceptance rate = 0.35452

# Stop Here.

## Check EPH effect

In [ ]:
dp = dpdm_delay.dpdm_block(model='U1B')

model0  = tm + wnb + dmn + spn + eph + dp
model1  = tm + wnb + dmn + spn + eph + dp + bdn
signal=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        signal.append( model1( psr ) )
    else:
        signal.append( model0( psr ) )

PTA = signal_base.PTA(signal)
PTA.set_default_params(Dict)

In [ ]:
dp = dpdm_delay.dpdm_block(model='U1B')

model0  = tm + wnb + dmn + spn + eph 
model1  = tm + wnb + dmn + spn + eph + bdn
signal=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        signal.append( model1( psr ) )
    else:
        signal.append( model0( psr ) )

PTA_eph = signal_base.PTA(signal)
PTA_eph.set_default_params(Dict)



In [ ]:
outDir="bayes/"+"_m"+str(nmodes)+"_EPH"

xs = {par.name:par.sample() for par in PTA_eph.params}
for parname in Dict.keys():
    if parname in xs.keys():
        xs[parname] = Dict[parname]

x0 = np.hstack([xs[key] for key in sorted(xs.keys())])    
ndim = len(x0)
#============================

x0[0:11] = 0.

groups=[range(ndim)]

print(PTA_eph.get_lnlikelihood(x0))


covdiag = np.ones(ndim)*0.01

sampler = PTSampler(ndim, PTA_eph.get_lnlikelihood, PTA_eph.get_lnprior,
                cov=np.diag(covdiag),outDir=outDir,groups=groups,resume=False)

sampler.sample(x0, 100000, isave=1000)

## Additional part, MLH contribution by pulsar

In [ ]:
# original
dp = dpdm_delay.dpdm_block(model='U1B')


model0  = tm + wnb + dmn + spn + eph + dp
model1  = tm + wnb + dmn + spn + eph + dp + bdn
signal=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        signal.append( model1( psr ) )
    else:
        signal.append( model0( psr ) )

PTA = signal_base.PTA(signal)
PTA.set_default_params(Dict)

In [ ]:
model0  = tm + wnb + dmn + spn + dp + eph
model1  = tm + wnb + dmn + spn + dp + eph + bdn
ptas=[]
for psr in psrs:
    if psr.name in ['J0437-4715','J1939+2134']:
        pta = signal_base.PTA(model1(psr))
        pta.set_default_params(Dict)
        ptas.append(pta)
    else:
        pta = signal_base.PTA(model0(psr))
        pta.set_default_params(Dict)
        ptas.append(pta)
        
dp = dpdm_delay.dpdm_block(model='U1B')


In [ ]:
# load mlh results

u1b_n106 = np.load('bayes/u1b_n106.npy')[0][0]
u1b_n106 = PTA.map_params(u1b_n106)

u1b_n091 = np.load('bayes/u1b_n091.npy')[0][0]
u1b_n091 = PTA.map_params(u1b_n091)

u1b_n105 = np.load('bayes/u1b_n105.npy')[0][0]
u1b_n105 = PTA.map_params(u1b_n105)

u1b_n102 = np.load('bayes/u1b_n102.npy')[0][0]
u1b_n102 = PTA.map_params(u1b_n102)

Null = {par.name:0 for par in PTA.params}
Null['x_dp_log10_eps']=-80

In [ ]:
print(PTA.get_lnlikelihood(u1b_n091),PTA.get_lnlikelihood(Null))

mlhs1=[]
for pta in ptas:
    mlhs1.append(pta.get_lnlikelihood(u1b_n106)-pta.get_lnlikelihood(Null) )
print(mlhs1)


mlhs2=[]
for pta in ptas:
    mlhs2.append(pta.get_lnlikelihood(u1b_n091)-pta.get_lnlikelihood(Null) )
print(mlhs2)

mlhs3=[]
for pta in ptas:
    mlhs3.append(pta.get_lnlikelihood(u1b_n105)-pta.get_lnlikelihood(Null) )
print(mlhs3)

mlhs4=[]
for pta in ptas:
    mlhs4.append(pta.get_lnlikelihood(u1b_n102)-pta.get_lnlikelihood(Null) )
print(mlhs4)

names=[psr.name for psr in psrs]
print(names)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(mlhs1,'.')
plt.plot(mlhs2,'.')
plt.plot(mlhs3,'.')
plt.plot(mlhs4,'.')

In [ ]:
print(names[0],names[4],names[20],names[21],names[24],names[25])

In [ ]:
mlhs1=[]
for pta in ptas:
    mlhs1.append(pta.get_lnlikelihood(u1b_n106)-pta.get_lnlikelihood(Null) )
print(mlhs1)

In [2]:
!jupyter nbconvert --to python bayes_limit.ipynb

[NbConvertApp] Converting notebook bayes_limit.ipynb to python
[NbConvertApp] Writing 8217 bytes to bayes_limit.py
